# Exercício 7

Uma Família está para se mudar de cidade. Para tanto, ela contratou uma
transportadora que colocou a sua disposição um caminhão-baú com capacidade
volumétrica de 30m3. Infelizmente, não há como transportar todos os pertences
da família, de modo que ficou resolvido que será adotado uma estratégia que
minimize o prejuízo total ao se desfazer dos bens que não serão transportados.
Na tabela a seguir estão os dados levantados pela família. Desconsiderando o
aspecto de arranjo da carga no compartimento do caminhão, determinar quais os
bens que deverão ser transportados

|  **Descrição do Item**  	| **Volume(m³)** 	| **Prejuízo (R$)** 	|
|:-----------------------:	|:--------------:	|:-----------------:	|
|      **Sala Estar**     	|        8       	|        6000       	|
|     **Sala Jantar**     	|        7       	|        8000       	|
| **Dormitório do Casal** 	|        9       	|       10000       	|
| **Dormitório do filho** 	|        6       	|        5000       	|
|       **Cozinha**       	|        5       	|        6000       	|

# Solução 

## Modelo Matemático Formal

$$
\begin{align*}
\text{Minimizar} Z &= 6000(1-x_1) + 8000(1-x_2) + 10000(1-x_3) + 5000(1-x_4) + 6000(1-x_5) \\[10pt]
\text{Sujeito a:} \\
8x_1 + 7x_2 + 9x_3 + 6x_4 + 5x_5 &\leq 30 \\[5pt]
x_i &\in \{0,1\} \quad \forall i \in \{1,\ldots,5\}
\end{align*}
$$

$$
\begin{align*}
\text{Parâmetros:} \\[5pt]
v_i &= \text{volume do item } i \\
p_i &= \text{prejuízo caso o item } i \text{ não seja transportado} \\
C &= \text{capacidade do caminhão (30m³)} \\[5pt]
\text{onde } &i \in \{1,\ldots,5\}
\end{align*}
$$

In [1]:
# --- Imports --- #
import pyomo.environ as pyo
import pandas as pd
import numpy as np

In [5]:
# --- Declaração dos dados de entrada --- # 
lista_ambientes = ['SalaEstar', 'SalaJantar', 'DormCasal', 'DormFilho', 'Cozinha']
# --
volumes = {'SalaEstar': 8, 
           'SalaJantar': 7, 
           'DormCasal': 9, 
           'DormFilho': 6, 
           'Cozinha': 5}
# --
prejuizos = {'SalaEstar': 6000, 
             'SalaJantar': 8000, 
             'DormCasal': 10000, 
             'DormFilho': 5000, 
             'Cozinha': 6000}
# --
capacidade_caminhao = 30

In [27]:
# --- Declaração do Modelo Matemático --- #
modelo = pyo.ConcreteModel()

In [28]:
# --- Declaração dos Conjuntos de Iteração --- #
modelo.ambientes = pyo.Set(initialize=lista_ambientes)

In [29]:
# --- Declaração das Variáveis de Decisão --- #
modelo.x = pyo.Var(modelo.ambientes, domain=pyo.Binary)

In [30]:
# --- Declaração dos Parâmetros do Modelo --- #
modelo.capacidade_caminhao = pyo.Param(initialize=capacidade_caminhao)
# --
modelo.volumes = pyo.Param(modelo.ambientes, initialize=volumes)
# --
modelo.prejuizos = pyo.Param(modelo.ambientes, initialize=prejuizos)

In [31]:
# --- Declaração da Função Objetivo --- #
def funcao_objetivo(modelo):
    return sum(modelo.prejuizos[i] * (1 - modelo.x[i]) for i in modelo.ambientes)
modelo.objetivo = pyo.Objective(rule=funcao_objetivo, sense=pyo.minimize)

In [32]:
# --- Declaração das restrições do modelo --- #
def rest_volume(modelo):
    return sum(modelo.volumes[i]*modelo.x[i] for i in modelo.ambientes) <= modelo.capacidade_caminhao
modelo.rest_volume = pyo.Constraint(rule=rest_volume)

In [33]:
# --- Declaração do Solver --- #
solver = pyo.SolverFactory('gurobi')
resultados = solver.solve(modelo)

In [34]:
# --- Impressão dos Resultados --- #
print('\nResultados:')
print('Status:', resultados.solver.status)
print('Condição de parada:', resultados.solver.termination_condition)
print('\nDecisões:')
volume_total = 0
items_transportados = []
# --
for i in modelo.ambientes:
    if modelo.x[i].value == 1:
        items_transportados.append(i)
        volume_total += volumes[i]
        print(f'Item {i}: Tranportar')
    else:
        print(f'Item {i}: Não Transportar')
# --
print('\nVolume Total Transportado:', volume_total, 'm³')
print('Prejuizo Total: R$', modelo.objetivo())


Resultados:
Status: ok
Condição de parada: optimal

Decisões:
Item SalaEstar: Tranportar
Item SalaJantar: Tranportar
Item DormCasal: Tranportar
Item DormFilho: Não Transportar
Item Cozinha: Tranportar

Volume Total Transportado: 29 m³
Prejuizo Total: R$ 5000.0
